### Installation des dépendances

In [48]:
%pip install opencv-python
%pip install numpy
%pip install anytree
import numpy as np
import cv2 as cv
from anytree import Node, RenderTree, PreOrderIter, AsciiStyle
import copy
import os
import time

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Question 2 compression avec LZW

### Lecture des fichiers images

In [49]:
# Read image
shapeArray = []
imageInStringArray = []
imageInString = []
dataPath = "/data TP1/images/"
currDirectory = os.getcwd()
fileList = os.listdir(currDirectory + dataPath)

for file in fileList:
    filePath = currDirectory + dataPath + file
    img = cv.imread(filePath, cv.IMREAD_COLOR)
    if img is not None:
        # Get the dimensions of the image
        shapeArray += [img.shape]
        height, width, channels = img.shape
        # Reshape the image array to a 2D array where each row represents a pixel
        reshaped_img = img.reshape((height * width, channels))
        # Convert the reshaped array to an array of string
        imageInString = [str(char) for char in reshaped_img.flatten()]
    imageInStringArray += [imageInString]
    
print(shapeArray)

[(255, 255, 3), (256, 512, 3), (476, 458, 3), (500, 500, 3), (379, 430, 3)]


### Lecture des fichiers texte

In [50]:
# Read txt files
messageList = []
dataPath = "/data TP1/textes/"
currDirectory = os.getcwd()
fileList = os.listdir(currDirectory + dataPath)
for file in fileList:
    filePath = currDirectory + dataPath + file
    if file.endswith(".txt"):
        with open(filePath, 'r') as f:
            messageList += [f.read()]
messageList += imageInStringArray            

### Fonction qui effectue le codage LZW

In [51]:
def encode(message : str):
    dictsymb =[message[0]]
    dictbin = ["{:b}".format(0)]
    nbsymboles = 1
    for i in range(1,len(message)):
        if message[i] not in dictsymb:
            dictsymb += [message[i]]
            dictbin += ["{:b}".format(nbsymboles)] 
            nbsymboles +=1       
    for i in range(nbsymboles):
        dictbin[i] = "{:b}".format(i).zfill(int(np.ceil(np.log2(nbsymboles))))  
        dictsymb.sort()
    basicdictsymb = copy.deepcopy(dictsymb)
    basicdictbin = copy.deepcopy(dictbin)
        
    #encode
    i=0
    messageCode = []
    longueur = 0
    while i < len(message):
        precsouschaine = message[i] #substring to be coded
        souschaine = message[i] #substring to be coded + 1 character (for the dictionary)

        # Look for the longest substring. We add one character at a time.
        while souschaine in dictsymb and i < len(message):
            i += 1
            precsouschaine = souschaine
            if i < len(message):  #If we are not at the end of the message
                souschaine += message[i]  

        # Coding of the longest substring using the dictionary
        codebinaire = [dictbin[dictsymb.index(precsouschaine)]]
        messageCode += codebinaire
        longueur += len(codebinaire[0]) 
        # Add encoded substring + next symbol in dictionary
        if i < len(message):
            dictsymb += [souschaine]
            dictbin += ["{:b}".format(nbsymboles)] 
            nbsymboles +=1

        #Add 1 bit if required
        if np.ceil(np.log2(nbsymboles)) > len(messageCode[-1]):
            for j in range(nbsymboles):
                dictbin[j] = "{:b}".format(j).zfill(int(np.ceil(np.log2(nbsymboles))))
    return messageCode, basicdictsymb, basicdictbin

Le code de décodage ci-dessous permet de décoder un message(string)  codé avec LZW il est aussi fonctionnel pour les images.
On ne recrée pas l'image mais on garantit que la chaîne de caractère reconstituée correspond à celle fournie en paramètre

In [52]:
def decode(message : str, basicdictsymb, basicdictbin):
    dictsymb = basicdictsymb
    dictbin = basicdictbin

    # decode the message
    i=0
    decodedMessage = ""
    nbsymboles = len(basicdictsymb)
    nbBitUsed = int(np.ceil(np.log2(nbsymboles)))
    while i < len(message):
        currSymb = dictsymb[dictbin.index(message[i])]
        decodedMessage += currSymb
        # add the new symbol to the dictionary
        nbsymboles += 1
        nbBitUsed = int(np.ceil(np.log2(nbsymboles)))
        dictbin += ["{:b}".format(nbsymboles)]
        # add 1bit if required
        if 2**nbBitUsed > len(dictsymb):
            for j in range(nbsymboles):
                dictbin[j] = "{:b}".format(j).zfill(int(np.ceil(np.log2(nbsymboles))))
        if i+1 < len(message):
            try:
                nextSymb = dictsymb[dictbin.index(message[i+1])]
            except:
                nextSymb = currSymb

        dictsymb += [currSymb+nextSymb[0]]
        i+=1
    return decodedMessage



# Question 3 implémentation d'un autre algorithme : HUFFMAN

In [70]:
def huffmanEncoding(message : str) -> str:
    ArbreSymb =[[message[0], message.count(message[0]), Node(message[0])]] 
    # dict got from the tree
    dictionnaire = [[message[0], '']]
    nbsymboles = 1

    # Search for the leaves of the tree
    for i in range(1,len(message)):
        if not list(filter(lambda x: x[0] == message[i], ArbreSymb)):
            ArbreSymb += [[message[i], message.count(message[i]),Node(message[i])]]
            dictionnaire += [[message[i], '']]
            nbsymboles += 1
  
    ArbreSymb = sorted(ArbreSymb, key=lambda x: x[1])
    

    while len(ArbreSymb) > 1:
        # Fusion of the two nodes with the lowest weight
        symbfusionnes = ArbreSymb[0][0] + ArbreSymb[1][0] 
        # Create a new node
        noeud = Node(symbfusionnes)
        temp = [symbfusionnes, ArbreSymb[0][1] + ArbreSymb[1][1], noeud]
        # Connect the new node to its parents
        ArbreSymb[0][2].parent = noeud
        ArbreSymb[1][2].parent = noeud
        # Remove the merged nodes from the list of nodes to be merged.
        del ArbreSymb[0:2]
        # Add the new node to the list and sort it.
        ArbreSymb += [temp]
        ArbreSymb = sorted(ArbreSymb, key=lambda x: x[1])  

    ArbreCodes = Node('')
    noeud = ArbreCodes
    parcoursprefix = [node for node in PreOrderIter(ArbreSymb[0][2])]
    parcoursprefix = parcoursprefix[1:len(parcoursprefix)] #ignore root

    Prevdepth = 0 #follow depth of the tree
    for node in parcoursprefix:  # Node list 
        if Prevdepth < node.depth: # We go deeper in the tree, so we put a 0
            temp = Node(noeud.name + '0')
            noeud.children = [temp]
            if node.children: # Advance the "pointer" if the added node has children.
                noeud = temp
        elif Prevdepth == node.depth: # Same depth, so we put a 1
            temp = Node(noeud.name + '1')
            noeud.children = [noeud.children[0], temp]  # Add second child
            if node.children: # Advance the "pointer" if the added node has children.
                noeud = temp
        else:
            for i in range(Prevdepth-node.depth): # Another branch so we put 1
                noeud = noeud.parent #OWe go back to the parent node to take another branch
            temp = Node(noeud.name + '1')
            noeud.children = [noeud.children[0], temp]
            if node.children:
                noeud = temp

        Prevdepth = node.depth    

    ArbreSymbList = [node for node in PreOrderIter(ArbreSymb[0][2])]
    ArbreCodeList = [node for node in PreOrderIter(ArbreCodes)]

    for i in range(len(ArbreSymbList)):
        if ArbreSymbList[i].is_leaf: # Generate leaf dictionary
            temp = list(filter(lambda x: x[0] == ArbreSymbList[i].name, dictionnaire))
            if temp:
                indice = dictionnaire.index(temp[0])
                dictionnaire[indice][1] = ArbreCodeList[i].name

    messageCode = []
    longueur = 0 
    for i in range(len(message)):
        substitution = list(filter(lambda x: x[0] == message[i], dictionnaire))
        messageCode += [substitution[0][1]]
        longueur += len(substitution[0][1]) 
    
    return messageCode

### Fonction utilisée pour afficher les taux de compression et le temps d'exécution on passe en paramètre le type de l'algorithme à exécuter

In [54]:
from enum import Enum
class Algo(Enum):
    LZW = 1
    HUFFMAN = 2
def printCompressionStat(algo : Algo) :
    compressionRatio = []
    executionTime = []
    encodeFunction = encode if algo == Algo.LZW else huffmanEncoding
    for message in messageList:
        startTime = time.time()
        encodedMessage = encodeFunction(message)
        endTime = time.time()
        encodedMessage = encodedMessage[0] if algo == Algo.LZW else encodedMessage
        compressionRatio += [ 1 - (len(encodedMessage[0])/len(message))]
        executionTime += [endTime - startTime]
    print("Taux de compression : \t", compressionRatio)
    print("Temps d'exécution : \t", executionTime)

### Test pour vérifier que le décodage est fonctionnel en LZW

In [55]:
allTestsPassed = True
# Not all photos are selected due to running time
testMessage = messageList[0:4] + [messageList[6]]
for message in testMessage:
    encodedMessage = encode(message)
    decodedMessage = decode(encodedMessage[0], encodedMessage[1], encodedMessage[2])
    messageInString = "".join(message)
    allTestsPassed = allTestsPassed and (decodedMessage == messageInString)

if(allTestsPassed):
    print("Le décodage est correct pour tous les messages.")
else:
    print("Le décodage n'est pas correct pour tous les messages.")


Le décodage est correct pour tous les messages.


### Afficher les résultats de la compression avec LZW

In [56]:
printCompressionStat(Algo.LZW)
    
    

Taux de compression : 	 [0.9880952380952381, 0.974025974025974, 0.9215686274509804, 0.9725274725274725, 0.9954988747186797, 0.9999589901320005, 0.9999974568684896, 0.9999877680329774, 0.9999893333333333, 0.9999897731688859]
Temps d'exécution : 	 [0.0, 0.0005142688751220703, 0.0, 0.0007731914520263672, 0.0063936710357666016, 62.572556257247925, 0.6318004131317139, 340.4256331920624, 872.3666620254517, 2.071165084838867]


### Afficher les résultats de la compression avec Huffman

In [57]:
printCompressionStat(Algo.HUFFMAN)

Taux de compression : 	 [0.9880952380952381, 0.974025974025974, 0.9215686274509804, 0.9615384615384616, 0.9932483120780196, 0.9999846212995002, 1.0, 0.9999984710041222, 0.9999906666666667, 0.9999979546337772]
Temps d'exécution : 	 [0.0005064010620117188, 0.0, 0.0005152225494384766, 0.0010251998901367188, 0.0335383415222168, 4.9135801792144775, 0.21778416633605957, 16.087618350982666, 18.710458755493164, 1.6999592781066895]
